In [ ]:
# follows tutorial found here
# https://cloud.google.com/bigquery/docs/making-predictions-with-sklearn-models-in-onnx-format

In [ ]:
# other important gifts
# https://onnx.ai/sklearn-onnx/api_summary.html

# Installs

In [ ]:
! pip install xgboost -U -q --user
! pip install skl2onnx -U -q --user

# Setup

In [2]:
P = ! gcloud config list --format 'value(core.project)'
PROJECT_ID = P[0]
PROJECT_NUMBER = !gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = PROJECT_NUMBER[0]
REGION = "us-central1"

# raw source data
BUCKET_NAME = f"bkt-{REGION}-data"
BUCKET_PATH = f"gs://{BUCKET_NAME}"
USE_CASE = "bq_inference_engine"

# model
MODEL_NAME = "calibration_model"

# BQ
BQ_DATASET = "ds_uscentral1"
BQ_TABLE = "calibration_test_set"
BQ_MODEL_NAME = f"bq_{MODEL_NAME}"

# Train an XGBoost classification model with an scikit-learn calibration model

In [3]:
# Import necessary libraries
from sklearn import datasets
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import brier_score_loss
import pandas as pd
import numpy as np
import json

In [36]:
# Load the Breast Cancer dataset
data = datasets.load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

# Add an ID column
X['ID'] = range(len(X))

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Drop 'ID' column from train and test sets for model training and scoring
X_train_ids = X_train.pop('ID')
X_test_ids = X_test.pop('ID')

# Train an XGBoost model
model = XGBClassifier(eval_metric='logloss')
model.fit(X_train, y_train)

# Predict probabilities for the test set
predicted_probabilities = model.predict_proba(X_test)

# Save probabilities and true outcomes to a new line delimited json file
test_set = [{'ID': int(myid), 'testds_xgb_pred_prob': round(float(prob[1]), 5), 'testds_label': int(y)} 
            for myid, prob, y in zip(X_test_ids, predicted_probabilities, y_test)]

# Platt Scaling on probabilties of positive class
positive_class_predicted_probabilties = predicted_probabilities[:, 1].reshape(-1, 1)
lr = LogisticRegression()
lr.fit(positive_class_predicted_probabilties, y_test)

# Calibrated probabilities
calibrated_probs = lr.predict_proba(positive_class_predicted_probabilties)

# Convert calibrated probabilities to DataFrame and add 'ID' column
calibrated_probs_df = pd.DataFrame(calibrated_probs[:, 1], columns=['xgb_calibrated_prob_nb'])
calibrated_probs_df['ID'] = X_test_ids.values
calibrated_probs_df = calibrated_probs_df.sort_values('ID')

# Print the before and after calibration Brier scores
print('Brier score before calibration:', brier_score_loss(y_test, predicted_probabilities[:, 1]))
print('Brier score after calibration:', brier_score_loss(y_test, calibrated_probs[:, 1]))

Brier score before calibration: 0.017434884531435425
Brier score after calibration: 0.020933944128659076


# Write test data partition to GCS -> BQ 
## (to verify ONNX model in BQ provides same results)

In [37]:
# write out
calibration_test_set_name = f"{BQ_TABLE}.json"
with open(calibration_test_set_name, 'w') as f:
    for item in test_set:
        f.write(json.dumps(item) + "\n")

print(f"Example row: {item}")
print('')

# save to GCS
calibration_test_set_uri = f"{BUCKET_PATH}/{USE_CASE}/{calibration_test_set_name}"
! gsutil cp {calibration_test_set_name} {calibration_test_set_uri}

Example row: {'ID': 247, 'testds_xgb_pred_prob': 0.99025, 'testds_label': 1}

Copying file://calibration_test_set.json [Content-Type=application/json]...
/ [1 files][ 10.6 KiB/ 10.6 KiB]                                                
Operation completed over 1 objects/10.6 KiB.                                     


In [38]:
def load_to_bq(PROJECT_ID, REGION, BQ_DATASET, BQ_TABLE, GCS_URI):
    from google.cloud import bigquery

    # Construct a BigQuery client object.
    client = bigquery.Client(location=REGION, project=PROJECT_ID)

    # TODO(developer): Set table_id to the ID of the table to create.
    table_id = f"{PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}"

    job_config = bigquery.LoadJobConfig(
        autodetect=True,
        source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON, )

    load_job = client.load_table_from_uri(
        GCS_URI,
        table_id,
        location=REGION,
        job_config=job_config, )

    assert load_job.job_type == "load"

    load_job.result()  # Waits for the job to complete.
    print('Job finished.')

    assert load_job.state == "DONE"
    destination_table = client.get_table(table_id)
    print('Loaded {} rows.'.format(destination_table.num_rows))
    
# Load to BQ
load_to_bq(PROJECT_ID, REGION, BQ_DATASET, BQ_TABLE, calibration_test_set_uri)

Job finished.
Loaded 171 rows.


# Convert the probability calibration model into ONNX format and save

In [ ]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

In [40]:
# Disable zipmap as it is not supported in BigQuery ML.
options = {id(lr): {'zipmap': False}}

# Define input features. scikit-learn does not store information about the
# training dataset. It is not always possible to retrieve the number of features
# or their types. That's why the function needs another argument called initial_types. Example

# define initial types
initial_types = [('testds_xgb_pred_prob', FloatTensorType([None, 1]))]

# Convert the model.
model_onnx = convert_sklearn(
   lr, MODEL_NAME, initial_types=initial_types, options=options
    , target_opset=17 # if not set, uses 18 which is unsupported
)

# Save the calibration model
calibration_model_name = f"{MODEL_NAME}.onnx"
with open(calibration_model_name, 'wb') as f:
    f.write(model_onnx.SerializeToString())

# Upload the ONNX model to Cloud Storage

In [42]:
calibration_model_uri = f"{BUCKET_PATH}/{USE_CASE}/{calibration_model_name}"
! gsutil cp {calibration_model_name} {calibration_model_uri}

Copying file://calibration_model.onnx [Content-Type=application/octet-stream]...
/ [1 files][  487.0 B/  487.0 B]                                                
Operation completed over 1 objects/487.0 B.                                      


# Import the ONNX model into BigQuery

In [43]:
def create_inference_engine_model(PROJECT_ID, REGION, BQ_DATASET, BQ_MODEL_NAME, GCS_URI):
    from google.cloud import bigquery

    # Construct a BigQuery client object.
    client = bigquery.Client(location=REGION, project=PROJECT_ID)

    # Create Remote Model In BigQuery
    query = f"""
    CREATE OR REPLACE MODEL `{PROJECT_ID}.{BQ_DATASET}.{BQ_MODEL_NAME}`
        OPTIONS (MODEL_TYPE='ONNX', MODEL_PATH='{GCS_URI}')
    """
    job = client.query(query = query)
    job.result()
    job.state
    
create_inference_engine_model(PROJECT_ID, REGION, BQ_DATASET, BQ_MODEL_NAME, calibration_model_uri)

# Make predictions with the imported ONNX model

In [52]:
def perform_inference_calibration_model(PROJECT_ID, REGION, BQ_DATASET, BQ_MODEL_NAME, BQ_TABLE):
    from google.cloud import bigquery

    # Construct a BigQuery client object.
    client = bigquery.Client(location=REGION, project=PROJECT_ID)

    # Create Remote Model In BigQuery
    query = f"""
    SELECT 
        ID
        , label AS ONNX_label
        , probabilities[offset(1)] AS ONNX_calibrated_prob_pos_class
        , testds_label
        , testds_xgb_pred_prob
    FROM ML.PREDICT(MODEL {BQ_DATASET}.{BQ_MODEL_NAME},
     (
      SELECT * FROM {PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}
     )
    )
    """
    #print("Query to run in console:")
    #print(query)
    job = client.query(query = query)
    df =job.to_dataframe().sort_values('ID')
    return df
    
df = perform_inference_calibration_model(PROJECT_ID, REGION, BQ_DATASET, BQ_MODEL_NAME, BQ_TABLE)
df.head(10)

,ID,ONNX_label,ONNX_calibrated_prob_pos_class,testds_label,testds_xgb_pred_prob
49,0,0,0.098023,0,0.01165
19,2,0,0.093009,0,0.00027
13,6,0,0.092957,0,0.00015
32,9,0,0.096496,0,0.00824
12,10,0,0.132607,0,0.07856
24,11,0,0.093168,0,0.00064
44,15,0,0.093462,0,0.00132
53,18,0,0.093013,0,0.00028
51,22,0,0.098787,0,0.01334
4,29,0,0.192781,0,0.16601


In [55]:
print("Below are the probabilties that were calculated in the notebook.")
print("You can compare these to the 'ONNX_calibrated_prob_pos_class' above to")
print("verify they are the same to 4 significant digits")
print('')
calibrated_probs_df.head(10)

Below are the probabilties that were calculated in the notebook.
You can compare these to the 'ONNX_calibrated_prob_pos_class' above to
verify they are the same to 4 significant digits



,xgb_calibrated_prob_nb,ID
130,0.098023,0
61,0.093009,2
38,0.092958,6
86,0.096494,9
36,0.132607,10
71,0.093167,11
119,0.093463,15
141,0.093013,18
135,0.098788,22
7,0.192778,29
